Lage en rad per person for å kunne kjøre tradisjonelle metoder for å analysere LGD

In [ ]:
%load_ext autoreload
%autoreload 2

figsize=(14, 4)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import file, plot, data, stat
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
pd.set_option('display.max_columns', None)
data_folder = os.path.join('../../..', 'data/prod')
file_name_collection = "Collection_data.csv"
file_path_collection = os.path.join(data_folder, file_name_collection)
file_name_konto = "konto_data_trimmed.csv"
file_path_konto = os.path.join(data_folder, file_name_konto)

In [ ]:
konto_frame, collection_frame = file.load_konto_data(
    file_path_konto
), file.load_collection_data(file_path_collection)
konto_frame = data.add_relevant_collectionid_to_konto_frame(
    konto_frame, collection_frame
)
konto_frame = data.reorder_column(konto_frame, "Collectionid", 3)
konto_frame = data.reorder_column(konto_frame, "CollectionOpenedAfter202309", 4)
konto_frame = data.reorder_column(konto_frame, "CollectionHasClosedDate", 5)

In [ ]:
konto_frame[konto_frame["PersonId"] == 1359].head(40)

In [ ]:
konto_frame_col_after_202309 = konto_frame.copy()#[konto_frame["CollectionOpenedAfter202309"] == True]
agg_def = {
    "PersonId": "last",
    "CollectionOpenedAfter202309": "last",
    "CollectionHasClosedDate": "last",
    "AgeGroup": "last",
    "Gender": "last",
    "Kommunenavn": "last",
    "MonthsSinceAccountCreatedNum": "last",
    "CreditLimitAmt": "last",
    "TurnoverAmt": "sum",
    "TurnoverNum": "sum",
    "TurnoverDomAmt": "sum",
    "TurnoverDomNum": "sum",
    "TurnoverIntAmt": "sum",
    "TurnoverIntNum": "sum",
    "FundtransferAmt": "sum",
    "FundtransferNum": "sum",
    "CashAtmAmt": "sum",
    "CashAtmNum": "sum",
    "CashCounterAmt": "sum",
    "CashCounterNum": "sum",
    "BalanceAmt": "mean",
    "OverdueAmt": "mean",
    "CreditLimitIncreaseFlag": "max",
    "MonthsInDCANum": "last",
    "Last_Airlines": "sum",
    "Last_Amusement and Entertainment": "sum",
    "Last_Automobile / Vehicle Rental": "sum",
    "Last_Business Services": "sum",
    "Last_Clothing Stores": "sum",
    "Last_Contracted Services": "sum",
    "Last_Government Services": "sum",
    "Last_Hotels": "sum",
    "Last_Includes all lodging merchants": "sum",
    "Last_Mail Order / Telephone Order Providers": "sum",
    "Last_Miscellaneous Stores": "sum",
    "Last_Others": "sum",
    "Last_Professional Services and Membership Organizations": "sum",
    "Last_Repair Services": "sum",
    "Last_Retail Stores": "sum",
    "Last_Service Providers": "sum",
    "Last_Transportation": "sum",
    "Last_Utilities": "sum",
    "Last_Wholesale Distributors and Manufacturers": "sum",
}
konto_frame_col_after_202309 = konto_frame_col_after_202309.groupby("Collectionid").agg(agg_def).reset_index()
cols_to_reverse = [
    "TurnoverAmt",
    "TurnoverDomAmt",
    "TurnoverIntAmt",
    "FundtransferAmt",
    "CashAtmAmt",
    "CashCounterAmt",
    "BalanceAmt",
    "OverdueAmt",
    "Last_Airlines",
    "Last_Amusement and Entertainment",
    "Last_Automobile / Vehicle Rental",
    "Last_Business Services",
    "Last_Clothing Stores",
    "Last_Contracted Services",
    "Last_Government Services",
    "Last_Hotels",
    "Last_Includes all lodging merchants",
    "Last_Mail Order / Telephone Order Providers",
    "Last_Miscellaneous Stores",
    "Last_Others",
    "Last_Professional Services and Membership Organizations",
    "Last_Repair Services",
    "Last_Retail Stores",
    "Last_Service Providers",
    "Last_Transportation",
    "Last_Utilities",
    "Last_Wholesale Distributors and Manufacturers",
]
for col in cols_to_reverse:
    konto_frame_col_after_202309[col] = konto_frame_col_after_202309[col] * -1
    konto_frame_col_after_202309[col] = konto_frame_col_after_202309[
        col
    ].replace(-0.0, 0.0)
cols_to_avg = [
    "Turnover",
    "TurnoverDom",
    "TurnoverInt",
    "Fundtransfer",
    "CashAtm",
    "CashCounter",
]
for col in cols_to_avg:
    konto_frame_col_after_202309[f"Avg{col}"] = konto_frame_col_after_202309[f"{col}Amt"] / konto_frame_col_after_202309[f"{col}Num"]
    konto_frame_col_after_202309.loc[
        konto_frame_col_after_202309[f"{col}Num"] == 0, f"Avg{col}"
    ] = 0

cols_to_share = [
    "TurnoverDomAmt",
    "TurnoverIntAmt",
    "FundtransferAmt",
    "CashAtmAmt",
    "CashCounterAmt",
    "Last_Airlines",
    "Last_Amusement and Entertainment",
    "Last_Automobile / Vehicle Rental",
    "Last_Business Services",
    "Last_Clothing Stores",
    "Last_Contracted Services",
    "Last_Government Services",
    "Last_Hotels",
    "Last_Includes all lodging merchants",
    "Last_Mail Order / Telephone Order Providers",
    "Last_Miscellaneous Stores",
    "Last_Others",
    "Last_Professional Services and Membership Organizations",
    "Last_Repair Services",
    "Last_Retail Stores",
    "Last_Service Providers",
    "Last_Transportation",
    "Last_Utilities",
    "Last_Wholesale Distributors and Manufacturers",
]
for col in cols_to_share:
    konto_frame_col_after_202309[f"{col}_share_of_Turnover"] = konto_frame_col_after_202309[col] / konto_frame_col_after_202309["TurnoverAmt"]
    konto_frame_col_after_202309.loc[
        konto_frame_col_after_202309["TurnoverAmt"] == 0, f"{col}_share_of_Turnover"
    ] = 0

cols_to_drop = [
    "TurnoverNum",
    "TurnoverDomAmt",
    "TurnoverDomNum",
    "TurnoverIntAmt",
    "TurnoverIntNum",
    "FundtransferAmt",
    "FundtransferNum",
    "CashAtmAmt",
    "CashAtmNum",
    "CashCounterAmt",
    "CashCounterNum",
    "Last_Airlines",
    "Last_Amusement and Entertainment",
    "Last_Automobile / Vehicle Rental",
    "Last_Business Services",
    "Last_Clothing Stores",
    "Last_Contracted Services",
    "Last_Government Services",
    "Last_Hotels",
    "Last_Includes all lodging merchants",
    "Last_Mail Order / Telephone Order Providers",
    "Last_Miscellaneous Stores",
    "Last_Others",
    "Last_Professional Services and Membership Organizations",
    "Last_Repair Services",
    "Last_Retail Stores",
    "Last_Service Providers",
    "Last_Transportation",
    "Last_Utilities",
    "Last_Wholesale Distributors and Manufacturers",
]
konto_frame_col_after_202309 = konto_frame_col_after_202309.drop(cols_to_drop, axis=1)
konto_frame_col_after_202309["CreditLimitIncreaseFlag"] = konto_frame_col_after_202309["CreditLimitIncreaseFlag"].map({0: False, 1: True})
konto_frame_col_after_202309["Gender"] = konto_frame_col_after_202309["Gender"].map({"F": False, "M": True})
konto_frame_col_after_202309["AgeGroup"] = konto_frame_col_after_202309["AgeGroup"].map(
    {"0 - 24": 0, "25 - 34": 1, "35 - 44": 2, "45 - 54": 3, "55 - 64": 4, "> 64": 5}
).astype(int)


konto_frame_col_after_202309.columns = konto_frame_col_after_202309.columns.str.replace(
    r"^Last_", "", regex=True
)
konto_frame_col_after_202309.rename(
    columns={"OverdueAmt": "AvgOverdueAmt"}, inplace=True
)
konto_frame_col_after_202309.rename(
    columns={"BalanceAmt": "AvgBalanceAmt"}, inplace=True
)
konto_frame_col_after_202309.rename(
    columns={"CreditLimitIncreaseFlag": "HasIncreasedCreditLimit"}, inplace=True
)
konto_frame_col_after_202309.rename(
    columns={"Gender": "Male"}, inplace=True
)

In [ ]:
collection_loss_frame = (
    collection_frame[["Collectionid", "CumulativeLossAmt"]]
    .groupby("Collectionid")
    .tail(1)
)
collection_loss_frame["LGD=0"] = (collection_loss_frame["CumulativeLossAmt"] == 0).astype(bool)

In [ ]:
konto_frame_prepared = konto_frame_col_after_202309.merge(collection_loss_frame[["Collectionid", "LGD=0"]], on="Collectionid", how="left")
konto_frame_prepared = konto_frame_prepared.set_index("Collectionid",drop=True) #?
konto_frame_prepared.sort_values(by="PersonId", ascending=True).head(30)

In [ ]:
ct = pd.crosstab(
    index=[
        konto_frame_prepared["CollectionOpenedAfter202309"],
        konto_frame_prepared["CollectionHasClosedDate"],
    ],
    columns=konto_frame_prepared["LGD=0"],
)

# Optional: rename rows for readability
ct.index.names = ["CollectionOpenedAfter202309", "CollectionHasClosedDate"]
ct.columns.name = "LGD=0"

ct_reset = ct.reset_index()

# Create a single label for each combination
ct_reset["Combo"] = (
    ct_reset["CollectionOpenedAfter202309"].astype(str)
    + " / "
    + ct_reset["CollectionHasClosedDate"].astype(str)
)

# Plot stacked bar chart
ct_reset.plot(
    x="Combo",
    kind="bar",
    stacked=True,
    color=["skyblue", "salmon"],  # 0 and 1 colors
    figsize=(8, 5),
)
plt.ylabel("Count of LGD=0")
plt.title("LGD=0 distribution by CollectionOpenedAfter202309 & CollectionHasClosedDate")
plt.xticks(rotation=45)
plt.show()
ct

## Actual modelling

In [ ]:
cols_to_drop = [
    "PersonId",
    "CollectionOpenedAfter202309",
    "CollectionHasClosedDate",
    "Kommunenavn",
    "LGD=0",
]
features = konto_frame_prepared.drop(columns=cols_to_drop)
target = konto_frame_prepared["LGD=0"]
features = features.apply(lambda x: x.astype(int) if x.dtype == "bool" else x)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, stratify=target)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
plot.confusion_matrix(y_pred, y_test, title="Confusion matrix for LGD=0 (False=0, True=1)")

In [ ]:
importances = rf.feature_importances_
feat_importance_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)
plt.figure(figsize=(8, 6))
plt.barh(
    feat_importance_df["Feature"], feat_importance_df["Importance"], color="skyblue"
)
plt.gca().invert_yaxis()  # highest importance on top
plt.xlabel("Importance")
plt.title("Feature Importances")
plt.show()

In [ ]:
y_proba = rf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color="blue", label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="grey", linestyle="--")  # random guess line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.show()